You are an expert Python developer specializing in the Databricks environment. Your task is to create a complete Python script to be executed within a Databricks notebook. The script must perform the following operations:
1.	Data Retrieval from SpaceX API:
o	Interact with the SpaceX v3 REST API (https://api.spacexdata.com/v3).
o	Retrieve data from one specific endpoint: 
	All launches: https://api.spacexdata.com/v3/launches
o	Handle potential errors during the API calls (e.g., timeouts, non-200 status codes).
2.	Filter Operation:
o	Perform a "filter" operation on the retrieved launch data based on specific criteria.
o	Filter Logic: Filter the list of launches based on launch year and launch success status. Only include launches that match the specified criteria.
o	The result should be a list of dictionaries, containing only the launch records that satisfy the filter conditions.
3.	Control Parameters and Debugging:
o	Include variables at the beginning of the script to define the API endpoint URL and the filtering criteria, making them easily modifiable. Use parameters like: 
	API_ENDPOINT_URL = "https://api.spacexdata.com/v3/launches"
	FILTER_YEAR = 2019 (Set to an integer year, or None to disable year filtering)
	FILTER_SUCCESS = True (Set to True, False, or None to disable success filtering)
o	Use Python's standard logging module to provide informative output during execution. Configure logging to display messages at the INFO level.
o	Log key messages such as: starting data retrieval, number of launches retrieved, filtering criteria being applied, number of launches before filtering, number of launches after filtering, starting upload to httpbin, upload outcome.
4.	Execution Time Measurement:
o	Code Execution Time: Measure the time taken to perform the main operations (data retrieval + filtering). Print this time after the filtering operation is complete.
o	Pipeline Execution Time: Measure the total execution time of the entire script (from the beginning until after the upload to httpbin). Print this total time at the end of the script. Use Python's time module.
5.	Upload Result:
o	Take the resulting filtered list from the filter operation.
o	Serialize it into JSON format.
o	Make an HTTP POST request to the https://httpbin.org/post endpoint, sending the resulting filtered JSON data in the request body.
o	Verify the response from httpbin.org (e.g., check the status code) and log the outcome of the upload operation.


In [0]:
import requests
import logging
import time
import json

# Control Parameters
API_ENDPOINT_URL = "https://api.spacexdata.com/v3/launches"
FILTER_YEAR = 2019  # Set to an integer year, or None to disable year filtering
FILTER_SUCCESS = True  # Set to True, False, or None to disable success filtering

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Measure total pipeline execution time
pipeline_start_time = time.time()

try:
    # Log start of data retrieval
    logging.info("Starting data retrieval from SpaceX API...")
    retrieval_start_time = time.time()

    # Make API call to retrieve launch data
    try:
        response = requests.get(API_ENDPOINT_URL, timeout=10)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
        launches = response.json()
        logging.info(f"Data retrieval successful. Retrieved {len(launches)} launches.")
    except requests.exceptions.RequestException as e:
        logging.error(f"Error during API call: {e}")
        raise

    retrieval_end_time = time.time()
    logging.info(f"Data retrieval completed in {retrieval_end_time - retrieval_start_time:.2f} seconds.")

    # Filter operation
    logging.info("Starting filtering operation...")
    filter_start_time = time.time()

    # Apply filtering criteria
    filtered_launches = []
    for launch in launches:
        include_launch = True

        # Apply year filter if FILTER_YEAR is set
        if FILTER_YEAR is not None and launch.get("launch_year") != str(FILTER_YEAR):
            include_launch = False

        # Apply success filter if FILTER_SUCCESS is set
        if FILTER_SUCCESS is not None and launch.get("launch_success") != FILTER_SUCCESS:
            include_launch = False

        if include_launch:
            filtered_launches.append(launch)

    filter_end_time = time.time()
    logging.info(f"Filtering completed in {filter_end_time - filter_start_time:.2f} seconds.")
    logging.info(f"Number of launches before filtering: {len(launches)}")
    logging.info(f"Number of launches after filtering: {len(filtered_launches)}")

    # Serialize filtered data to JSON
    filtered_data_json = json.dumps(filtered_launches)

    # Upload result to httpbin.org
    logging.info("Starting upload to httpbin.org...")
    upload_start_time = time.time()

    try:
        upload_response = requests.post("https://httpbin.org/post", data=filtered_data_json, timeout=10)
        upload_response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
        logging.info(f"Upload successful. Response status code: {upload_response.status_code}")
    except requests.exceptions.RequestException as e:
        logging.error(f"Error during upload to httpbin.org: {e}")
        raise

    upload_end_time = time.time()
    logging.info(f"Upload completed in {upload_end_time - upload_start_time:.2f} seconds.")

except Exception as e:
    logging.error(f"An error occurred during execution: {e}")

finally:
    # Measure total pipeline execution time
    pipeline_end_time = time.time()
    logging.info(f"Total pipeline execution time: {pipeline_end_time - pipeline_start_time:.2f} seconds.")

The code need a fix so i try to put the output in qwen again

In [0]:
import requests
import logging
import time
import json

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Control parameters (easily modifiable)
LAUNCHES_API_URL = "https://api.spacexdata.com/v3/launches"
ROCKETS_API_URL = "https://api.spacexdata.com/v3/rockets"
HTTPBIN_POST_URL = "https://httpbin.org/post"

def fetch_data(api_url):
    """
    Fetch data from a given API endpoint.
    Handles errors such as timeouts or non-200 status codes.
    """
    try:
        logging.info(f"Fetching data from: {api_url}")
        response = requests.get(api_url, timeout=10)
        response.raise_for_status()  # Raise an exception for HTTP errors
        return response.json()
    except requests.exceptions.RequestException as e:
        logging.error(f"Error fetching data from {api_url}: {e}")
        return None

def merge_launches_and_rockets(launches, rockets):
    """
    Merge launches and rockets data based on rocket_id.
    Adds rocket_name to each launch record.
    """
    logging.info("Starting merge operation...")
    
    # Create a dictionary for quick lookup of rocket names by rocket_id
    rocket_lookup = {}
    for rocket in rockets:
        try:
            rocket_id = rocket['id']
            rocket_name = rocket.get('name', 'Unknown Rocket')  # Use 'Unknown Rocket' as fallback
            rocket_lookup[rocket_id] = rocket_name
        except KeyError as e:
            logging.warning(f"Missing key in rocket data: {e}. Skipping this rocket.")
    
    # Enrich each launch record with the corresponding rocket name
    merged_data = []
    for launch in launches:
        try:
            rocket_id = launch.get('rocket', {}).get('rocket_id')
            rocket_name = rocket_lookup.get(rocket_id, 'Unknown Rocket')
            enriched_launch = {**launch, 'rocket_name': rocket_name}
            merged_data.append(enriched_launch)
        except KeyError as e:
            logging.warning(f"Missing key in launch data: {e}. Skipping this launch.")
    
    logging.info(f"Merge operation completed. Total records after merge: {len(merged_data)}")
    return merged_data

def upload_to_httpbin(data):
    """
    Serialize data to JSON and upload it to httpbin.org/post.
    Logs the outcome of the upload operation.
    """
    try:
        logging.info("Starting upload to httpbin.org...")
        headers = {'Content-Type': 'application/json'}
        response = requests.post(HTTPBIN_POST_URL, data=json.dumps(data), headers=headers, timeout=10)
        response.raise_for_status()
        
        logging.info(f"Upload successful. Status code: {response.status_code}")
        logging.debug(f"Response from httpbin: {response.text}")
    except requests.exceptions.RequestException as e:
        logging.error(f"Error during upload to httpbin.org: {e}")

def main():
    # Start measuring total pipeline execution time
    total_start_time = time.time()
    
    # Step 1: Retrieve data from SpaceX API
    logging.info("Starting data retrieval from SpaceX API...")
    start_time = time.time()
    
    launches = fetch_data(LAUNCHES_API_URL)
    rockets = fetch_data(ROCKETS_API_URL)
    
    if launches is None or rockets is None:
        logging.error("Data retrieval failed. Exiting script.")
        return
    
    # Log sample rocket data for inspection
    logging.info(f"Sample rocket data: {rockets[:2]}")
    
    logging.info(f"Number of launches retrieved: {len(launches)}")
    logging.info(f"Number of rockets retrieved: {len(rockets)}")
    
    # Measure data retrieval time
    data_retrieval_time = time.time() - start_time
    logging.info(f"Data retrieval completed in {data_retrieval_time:.2f} seconds.")
    
    # Step 2: Merge launches and rockets data
    start_time = time.time()
    merged_data = merge_launches_and_rockets(launches, rockets)
    
    # Measure merge operation time
    merge_time = time.time() - start_time
    logging.info(f"Merge operation completed in {merge_time:.2f} seconds.")
    
    # Step 3: Upload merged data to httpbin.org
    upload_to_httpbin(merged_data)
    
    # Measure total pipeline execution time
    total_execution_time = time.time() - total_start_time
    logging.info(f"Total pipeline execution time: {total_execution_time:.2f} seconds.")

if __name__ == "__main__":
    main()